## CIVE4200  Assignment 2, Question 4 - Extended Explanation

This notebook illustrates the use of the Python version of CAL86 (CAL90?) to solve Question 4 of Assignment 2 of CIVE4200, 2015/16.

The frame to solve is the following:

![Element](img/A2Q2-fig0.svg)

In [1]:
import PyCAL86
import numpy as np

Neglecting axial deformations (ie, assuming infinite axial stiffness), the element
degree-of-freedom numbering is as follows: ![Element](img/A2Q2-fig1.svg)
Note that the numbering starts at 0 (zero) **not** 1 (one).  This is to keep array indexing
consistent with the underlying Python array library.

$\newcommand{\mat}[1]{\left[\begin{matrix}#1\end{matrix}\right]}$
The local member stiffness matrix expresses the end forces as a function of the end displacements, thus:
$$
K = \frac{EI}{L}\mat{12/L^2 & 6/L & -12/L^2 & 6/L \\
                  6/L  &  4 & -6/L & 2\\
                  -12/L^2 & -6/L & 12/L^2 & -6/L \\
                  6/L & 2 & -6/L & 4}
$$

Use basic Python/numpy functionality to compute the stiffness matrix for the 4m beam (on the left):
(this is *completely* optional - it is not required to use CAL)

In [2]:
E = 200000.   # N/mm^2,
def getK(E,I,L):
    """Return the element stiffness matrix and EI in units of kN, m"""
    EI = E*I / 1E3 / 1E6     # convert to kN-m^2
    return np.mat([[  12/L**2,  6/L, -12/L**2,  6/L ],
                   [   6/L,      4,   -6/L,      2  ],
                   [ -12/L**2, -6/L,  12/L**2, -6/L ],
                   [   6/L,      2,   -6/L,      4  ]]) * (EI/L), EI

getK(E,351.E6,4.)   # W530x66, L=4m

(matrix([[ 13162.5,  26325. , -13162.5,  26325. ],
         [ 26325. ,  70200. , -26325. ,  35100. ],
         [-13162.5, -26325. ,  13162.5, -26325. ],
         [ 26325. ,  35100. , -26325. ,  70200. ]]), 70200.0)

#### 1. Member Stiffness Matrices
Now use CAL commands to input the stiffness matrices for all three members:

In [3]:
%%CAL86
C        NAME: JOSHUA WOODS
C        COURSE CIVE 4200
C
C QUESTION 2 FROM ASSIGNMENT 2 SOLVED USING CAL90 USING BASIC COMMANDS AND ADDK IGNORING AXIAL DEFORMATIONS
C
C PROBLEM
C
C STIFFNESS CALCULATION
C
C MEMBER 1 (HORIZONTAL ON LEFT , L=4 EI=70.2E3)
C
LOAD K1 R=4 C=4
13.16E3 26.33E3 -13.16E3 26.33E3
26.33E3 70.2E3 -26.67E3 35.1E3
-13.16E3 -26.67E3 13.16E3 -26.33E3
26.33E3 35.1E3 -26.33E3 70.2E3
PRINT K1

K1:
[[ 13160.  26330. -13160.  26330.]
 [ 26330.  70200. -26670.  35100.]
 [-13160. -26670.  13160. -26330.]
 [ 26330.  35100. -26330.  70200.]]


In [4]:
getK(E,351.E6,2.)   # W530x66, L=2m

(matrix([[ 105300.,  105300., -105300.,  105300.],
         [ 105300.,  140400., -105300.,   70200.],
         [-105300., -105300.,  105300., -105300.],
         [ 105300.,   70200., -105300.,  140400.]]), 70200.0)

In [5]:
%%CAL86
C
C MEMBER2 (HORIZONTAL ON RIGHT, L=2, EI=70.2E3)
C
LOAD K2 R=4 C=4
105.3E3 105.3E3 -105.3E3 105.3E3
105.3E3 140.4E3 -105.3E3 70.2E3
-105.3E3 -105.3E3 105.3E3 -105.3E3
105.3E3 70.2E3 -105.3E3 140.4E3
PRINT K2

K2:
[[ 105300.  105300. -105300.  105300.]
 [ 105300.  140400. -105300.   70200.]
 [-105300. -105300.  105300. -105300.]
 [ 105300.   70200. -105300.  140400.]]


In [6]:
getK(E,556.E6,5.)   # W460x113, L=5m

(matrix([[ 10675.2,  26688. , -10675.2,  26688. ],
         [ 26688. ,  88960. , -26688. ,  44480. ],
         [-10675.2, -26688. ,  10675.2, -26688. ],
         [ 26688. ,  44480. , -26688. ,  88960. ]]), 111200.0)

In [7]:
%%CAL86
C
C MEMBER 3 (VERTICAL COLUMN MEMBER, L=5, EI=111.2E3, THETA=90)
LOAD KK3 R=4 C=4
10.675E3 26.69E3 -10.675E3 26.69E3
26.69E3 88.96E3 -26.69E3 44.48E3
-10.675E3 -26.69E3 10.675E3 -26.69E3
26.69E3 44.48E3 -26.69E3 88.96E3
PRINT KK3

KK3:
[[ 10675.  26690. -10675.  26690.]
 [ 26690.  88960. -26690.  44480.]
 [-10675. -26690.  10675. -26690.]
 [ 26690.  44480. -26690.  88960.]]


In [8]:
%%CAL86
C
C TRANSFORMATION MATRIX FOR THETA=90
C
LOAD A1 R=4 C=4
1. 0. 0. 0.
0. 1. 0. 0.
0. 0. 1. 0.
0. 0. 0. 1.
PRINT A1

A1:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


Form $\left[KA3\right] = \left[KK3\right] \left[A1\right]$:

In [9]:
%%CAL86
C
C FORCE-DISPLACEMENT TRANFORMATION MATRIX
C
MULT KK3 A1 KA3
PRINT KA3

KA3:
[[ 10675.  26690. -10675.  26690.]
 [ 26690.  88960. -26690.  44480.]
 [-10675. -26690.  10675. -26690.]
 [ 26690.  44480. -26690.  88960.]]


Form $\left[K3\right] = \left[A1\right]^T\left[KA3\right] = \left[A1\right]^T \left[KK3\right] \left[A1\right]$:

In [10]:
%%CAL86
C
TRAN A1 A1T
MULT A1T KA3 K3
PRINT K3

K3:
[[ 10675.  26690. -10675.  26690.]
 [ 26690.  88960. -26690.  44480.]
 [-10675. -26690.  10675. -26690.]
 [ 26690.  44480. -26690.  88960.]]


#### 2. Numbering the Frame DOFs
At each joint, we number the DOFs corresponding to the DOF directions at
each attached member end (each member must have the
four DOFs shown in an earlier figure).   All DOFs are numbered, starting with 0 (zero); the unconstrained DOFs
are numbered consecutively before the constrained DOFs.

Note that DOF's 4, 7, and 8 are constrained because of the support conditions and 5 and 6 are
constrained because of the assumed infinite axial stiffness of beam M1 and column M3.  So 0 through 3 are
unconstrained DOFs and 4 through 8 are constrained DOFs.

![Element](img/A2Q2-fig2.svg)

In the following matrix, the first column is for member M1, and it specifies that global DOF's 4, 0, 5 and 1 correspond to its local DOFs 0, 1, 2 and 3:

In [11]:
%%CAL86
C
C DIRECT STIFFNESS ASSEMBLY OF GLOBAL STRUCTURAL STIFFNESS
C
C LOCAL-STRUCTURAL DOF RELATION OR LOCATION ARRAY
C
LOADI LM R=4 C=3
4 5 7
0 1 8
5 3 6
1 2 1
PRINT LM

LM:
[[4 5 7]
 [0 1 8]
 [5 3 6]
 [1 2 1]]


#### 3. Assemble the Global Stiffness Matrix

In [12]:
%%CAL86
C
ZERO K R=9 C=9
ADDK K K1 LM N=0
ADDK K K2 LM N=1
ADDK K K3 LM N=2
C
C ASSEMBLED GLOBAL STRUCTURAL STIFFNESS
PRINT K

K:
[[  70200.   35100.       0.       0.   26330.  -26670.       0.       0.       0.]
 [  35100.  299560.   70200. -105300.   26330.   78970.  -26690.   26690.   44480.]
 [      0.   70200.  140400. -105300.       0.  105300.       0.       0.       0.]
 [      0. -105300. -105300.  105300.       0. -105300.       0.       0.       0.]
 [  26330.   26330.       0.       0.   13160.  -13160.       0.       0.       0.]
 [ -26670.   78970.  105300. -105300.  -13160.  118460.       0.       0.       0.]
 [      0.  -26690.       0.       0.       0.       0.   10675.  -10675.  -26690.]
 [      0.   26690.       0.       0.       0.       0.  -10675.   10675.   26690.]
 [      0.   44480.       0.       0.       0.       0.  -26690.   26690.   88960.]]


#### 4. Input the Loads and Constrained Displacements

In [13]:
%%CAL86
C
C LOAD VECTOR WITH FIXED END FORCES
LOAD R R=9 C=1
-16.
12.
4.
-17.
0
0
0
0
0
PRINT R
C
C DISPLACEMENT VECTOR
ZERO D R=9 C=1

R:
[[-16.]
 [ 12.]
 [  4.]
 [-17.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]]


#### 5. Solve for Displacements and Member End Forces

In [14]:
%%CAL86
C  IN THE FOLLOWING, PS IS PARTITION SIZE (# OF UNCONSTRAINED DOFs)
PSOLVE K D R PS=4
PRINT D
PRINT R
C
C MEMBER FORCES W / FIXED END FORCES
C
MEMFRC K1 D LM F1 N=0
C MEMBER FORCES MEMBER 1
PRINT F1
C
MEMFRC K2 D LM F2 N=1
C MEMBER FORCES MEMBER 2
PRINT F2
C
MEMFRC KA3 D LM F3 N=2
C MEMBER FORCES MEMBER 3
PRINT F3

D:
[[-0.00019]
 [-0.00007]
 [-0.00044]
 [-0.00067]
 [ 0.     ]
 [ 0.     ]
 [ 0.     ]
 [ 0.     ]
 [ 0.     ]]
R:
[[-16.     ]
 [ 12.     ]
 [  4.     ]
 [-17.     ]
 [ -6.93081]
 [ 23.99629]
 [  1.88475]
 [ -1.88475]
 [ -3.14102]]
F1:
[[ -6.93081]
 [-16.     ]
 [  6.99629]
 [-11.71796]]
F2:
[[ 17.]
 [ 30.]
 [-17.]
 [  4.]]
F3:
[[-1.88475]
 [-3.14102]
 [ 1.88475]
 [-6.28204]]


In [15]:
%%CAL86
C
C ADD FIXED END FORCES TO MEMBER 2
C
LOAD FE1 R=4 C=1
24.
16.
24.
-16.
PRINT FE1
C
LOAD FE2 R=4 C=1
12.
4.
12.
-4.
PRINT FE2
C
ADD F1 FE1
ADD F2 FE2
PRINT F1
PRINT F2
C
C PROBLEM COMPLETE ALL MEMBER FORCES AND NODAL DISPLACEMENTS HAVE BEEN DETERMINED
C RETURN

FE1:
[[ 24.]
 [ 16.]
 [ 24.]
 [-16.]]
FE2:
[[ 12.]
 [  4.]
 [ 12.]
 [ -4.]]
F1:
[[ 17.06919]
 [  0.     ]
 [ 30.99629]
 [-27.71796]]
F2:
[[ 29.]
 [ 34.]
 [ -5.]
 [  0.]]
